In [ ]:

# to test whether object identity is preserved during pickling...

import pickle
import json
class A:
    def __init__(self, x):
        self.x = x
class B:
    def __init__(self, y, a:A):
        self.y = y
        self.a = a
a1 = A(1)
a2 = A(2)
b1 = B(100, a1)
b2 = B(200, a1)
b3 = B(300, a2)
b4 = B(400, a2)
c1 = [a1, a2, b1, b2, b3, b4]
foo = lambda x, i: [x[i]==a for a in [x[0], x[1], x[2].a, x[3].a, x[4].a, x[5].a]]
d1 = foo(c1, 0)
d2 = foo(c1, 1)
data = pickle.dumps(c1)
c2 = pickle.loads(data)
e1 = foo(c2, 0)
e2 = foo(c2, 1)
for x in [d1, e1, d2, e2]: print(x)
print(d1 == e1 and d2 == e2)


In [ ]:
from netifaces import interfaces, ifaddresses, AF_INET
for ifaceName in interfaces():
    addresses = [i['addr'] for i in ifaddresses(ifaceName).setdefault(AF_INET, [{'addr':'No IP addr'}] )]
    print ('%s: %s' % (ifaceName, ', '.join(addresses)))

In [ ]:
from netifaces import interfaces, ifaddresses, AF_INET
print(interfaces())
print(ifaddresses('en0').setdefault(AF_INET))
print(ifaddresses('en0').setdefault(AF_INET)[0]['addr'])

In [ ]:
class A:
    def __init__(self):
        pass
class B:
    def __init__(self):
        pass
class C:
    def __init__(self):
        pass

def u(a,b=None):
    return Union[a,b] if b else a

def s_u(lis):
    r = Union[lis[0], lis[1]]
    r = Union[r,lis[2]]
    return r
print('s_u', s_u([A,B,C]))

def s_u_2(lis):
    r = None
    if len(lis) >=1:
        for c in lis:
            r = Union[r,c]
    return r
print('s_u_2', s_u_2([]))
print('s_u_2', s_u_2([A]))
print('s_u_2', s_u_2([A,B]))
print('s_u_2', s_u_2([A,B,C]))

un = lambda a, b=None: Union[a,b] if b else a

print((lambda a, b: a + b)(1,2))
print(Union[A,B])
print(Union[A,Union[B]])

print(list(u(x for x in base.Action.__subclasses__())))
print(list((lambda a, b=None: Union[a,b] if b else a)(x for x in base.Action.__subclasses__())))

def subclass_union(Class):
    subclasses = Class.__subclasses__()
    binary_union = lambda a, b=None: Union[a,b] if b else a
    return binary_union(x for x in subclasses)
print(subclass_union(base.Action))


In [ ]:
from tempfile import NamedTemporaryFile, TemporaryDirectory
t = NamedTemporaryFile()
try:
    print(t.name)
except:
    print('invalid file method name')
t.close()
d = TemporaryDirectory()
try:
    print(d.name)
except:
    print('invalid directory method name')
d.cleanup()

In [ ]:
from tempfile import gettempdir
gettempdir()

In [ ]:
from whendo.core.action import Action
list(s for s in Action.__subclasses__())

In [ ]:
import pathlib
str(pathlib.Path().home())

In [ ]:
import os
[
        dirpath
        for dirpath, dirnames, filenames in os.walk('whendo')
        if os.path.exists(os.path.join(dirpath, "__init__.py"))
    ]

In [ ]:
# thoughts about wrapping error handling around action.execu
def during_period(self, action_execute:Callable[...,...], tag:str):
    if (self.start is not None) and (self.stop is not None):
        """
        is_in_period_wrapper below takes (start) and (stop) and returns a 1-arg function that compares (start)
        and (stop) with a supplied time (now).

        if start<stop, then the active period is defined by
            start<now AND now<stop
            e.g. - 6:00 => 18:00
        if start>stop, then the period period is defined by
            start<now OR now<stop
            e.g. - 18:00 => 6:00
        
        This is done so that is_in_period is freshly evaluated at the times when the schedule library runs the job
        (that is meant to invoke the Callable).
        """
        is_in_period_wrapper = lambda start, stop: (lambda now: start < now and now < stop if (start < stop) else start < now or now < stop)
        is_in_period = is_in_period_wrapper(self.start, self.stop)
        do_nothing = lambda tag, scheduler_info: None
        return self.wrap(lambda: action_execute if is_in_period(Now.t()) else do_nothing)(tag=tag, scheduler_info=self.info()))
    else:
        return self.wrap(lambda: action_execute(tag=tag, scheduler_info=self.info()))

def wrap(self, thunk:Callable[...,...]):
    result = None
    def execute(cls):
        try:
            result = thunk()
        except Exception as exception:
            result = exception
        return result
    return execute

In [ ]:
from whendo.core.action import Action
from pydantic import BaseModel
class X(BaseModel):
    x:str
    def y(self):
        self.x = 'z'

x = X(x='x')
y = x.y
y()
y.__self__.__class__

In [ ]:
dir(y)

In [ ]:
y.__self__.fields

In [ ]:
class Count:
    """
    This counter is useful for distiguishing between different sets of log entries as well as viewing a count for each tag.
    """
    good_count = {}
    bad_count = {}
    @classmethod
    def good_up(cls, tag):
        if tag not in cls.good_count:
            cls.good_count[tag] = 0
        cls.good_count[tag] = 1 + cls.good_count[tag]
    @classmethod
    def bad_up(cls, tag):
        if tag not in cls.bad_count:
            cls.bad_count[tag] = 0
        cls.bad_count[tag] = 1 + cls.bad_count[tag]
    @classmethod
    def good(cls, tag):
        return cls.good_count.get(tag, 0)
    @classmethod
    def bad(cls, tag):
        return cls.bad_count.get(tag, 0)
for i in range(0,1000000):
    Count.good_up('abc:def')
print(Count.good('abc:def'))

In [ ]:
from datetime import time
from whendo.sdk.client import Client
from whendo.core.scheduler import TimelyScheduler, RandomlyScheduler
from whendo.core.actions.gpio_action import TogglePin
from whendo.core.util import TimeUnit

# create action that toggles pin 25 (connected to red LED)
red_action = TogglePin(pin=25)
# create action that toggles pin 27 (connected to green LED)
green_action = TogglePin(pin=27)

# see how they toggle
# [ (lambda y: (red_action.execute(), green_action.execute(), time.sleep(y)))(x) for x in [1,2,3,4,3,2,1,2,3,4,3,2,1]]

# create a scheduler that executes an action every second
# second after the top of the minute, from 6:00 to 23:00.
# The start and stop times operate on a 24 hour clock.
timely_secondly = TimelyScheduler(interval=1)
randomly_secondly = RandomlyScheduler(interval=1, low=2, high=5, time_unit=TimeUnit.second)

# scheduled run...
client = Client()

client.add_action('green_action', green_action)
client.add_action('red_action', red_action)

client.add_scheduler('timely_secondly', timely_secondly)
client.add_scheduler('randomly_secondly', randomly_secondly)

client.schedule_action('timely_secondly', 'green_action')
client.schedule_action('randomly_secondly', 'red_action')
client.run_jobs()

# # change to pin 27
# red_toggle.pin = 27

# # since red_toggle is already scheduled, the system automatically
# #reschedules the action after setting to the new value.
# client.set_action('red_toggle', red_toggle)

# # run individually through the client
# client.execute_action('red_toggle')

# client.stop_jobs()

In [ ]:
client.stop_jobs()

In [ ]:
client.clear_dispatcher()

In [ ]:
import time
from whendo.sdk.client import Client
from whendo.core.scheduler import TimelyScheduler, RandomlyScheduler
from whendo.core.actions.gpio_action import TogglePin

# create action that toggles pin 25 (connected to red LED)
red_action = TogglePin(pin=25)
# create action that toggles pin 27 (connected to green LED)
green_action = TogglePin(pin=27)

In [ ]:
green_action.execute()

In [ ]:
red_action.execute()

In [ ]:
len([ (lambda y: (red_action.execute(), green_action.execute(), time.sleep(y)))(x) for x in [1,2,3,4,3,2,1,2,3,4,3,2,1] ])

In [ ]:
from datetime import time
import time as thyme
from whendo.sdk.client import Client
from whendo.core.scheduler import TimelyScheduler, RandomlyScheduler
from whendo.core.actions.gpio_action import TogglePin, SetPin, Cleanup
from whendo.core.util import TimeUnit

# create action that toggles pin 25 (connected to red LED)
red_toggle = TogglePin(pin=25)
red_off = SetPin(pin=red_toggle.pin, on=False)
# create action that toggles pin 27 (connected to green LED)
green_toggle = TogglePin(pin=27)
green_off = SetPin(pin=green_toggle.pin, on=False)

gpio_cleanup = Cleanup()

# create schedulers that executes an action every second from 18:00 to 8:00.
# Start/stop specify intervals 24 hours or less. The random one executes
# randomly from a period 2 to 5 seconds from the previous execution.
timely_secondly = TimelyScheduler(interval=1, start=time(18,0,0), stop=time(8,0,0))
randomly_secondly = RandomlyScheduler(interval=1, start=time(18,0,0), stop=time(6,0,0), low=2, high=5, time_unit=TimeUnit.second)

# SDK
# A client could be any reachable pi with whendo running on it
# with the same action libraries. Use a Jupyter notebook
# to interact with a fleet of pi's.
client = Client(host='127.0.0.1', port=8000)
client.clear_dispatcher() # deletes all actions and schedulers

client.add_action('red_toggle', red_toggle)
client.add_action('green_toggle', green_toggle)

client.add_scheduler('timely_secondly', timely_secondly)
client.add_scheduler('randomly_secondly', randomly_secondly)

client.schedule_action('timely_secondly', 'green_toggle')
client.schedule_action('randomly_secondly', 'red_toggle')

client.add_action('red_off', red_off)
client.add_action('green_off', green_off)
client.add_action('gpio_cleanup', gpio_cleanup)

client.run_jobs()
thyme.sleep(20)
client.stop_jobs()

# make sure the pins are turned off and GPIO cleaned up
client.execute_action('green_off')
client.execute_action('red_off')
client.execute_action('gpio_cleanup')
